In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import sys

In [2]:
distance_matrix = np.array(pd.read_excel("Book1.xlsx","distances", header=None))

In [3]:
city_names = np.array((pd.read_excel("Book1.xlsx","citynames",header = None)))

In [4]:
city_nodes = dict(zip(np.squeeze(city_names),[i for i in range(128)]))

In [5]:
key_list =list(city_nodes.keys())
node_list = list(city_nodes.values())

In [6]:
G = nx.Graph()

In [7]:
#adding nodes
G.add_nodes_from(np.squeeze(city_names))

In [8]:
#adding edges:
init_matrix = np.zeros((distance_matrix.shape[0],distance_matrix.shape[0]))
distance_btw = np.ones((distance_matrix.shape[0],distance_matrix.shape[0]))*sys.maxsize
for i in range(distance_matrix.shape[0]):
    for j in range(i):
        if distance_matrix[i][j]<=100:
            G.add_edge(key_list[node_list.index(i)],key_list[node_list.index(j)])
            G[key_list[node_list.index(i)]][key_list[node_list.index(j)]]["d"]=distance_matrix[i][j]
            init_matrix[i][j]=1
            init_matrix[j][i]=1
            distance_btw[i][j]=distance_matrix[i][j]
            distance_btw[j][i]=distance_matrix[i][j]
row_values = np.sum(init_matrix, axis =1)
col_values = np.sum(init_matrix, axis = 0)
rv_zip = list(zip(row_values,[i for i in range(128)]))
cv_zip = list(zip(col_values,[i for i in range(128)]))
zero_values = set([j for i,j in rv_zip if i==0])
zero_col = set([j for i,j in cv_zip if i ==0])
get_edge = (zero_values.intersection(zero_col))    
for i in get_edge:
    idx = np.argsort(distance_matrix[i])
    G.add_edge(key_list[node_list.index(i)],key_list[node_list.index(idx[1])])
    G[key_list[node_list.index(i)]][key_list[node_list.index(idx[1])]]["d"]=distance_matrix[i][idx[1]]
    distance_btw[i][idx[1]]=distance_matrix[i][idx[1]]
    distance_btw[idx[1]][i]= distance_matrix[i][idx[1]]

# Dijkstra's algorithm

In [9]:
def dijstra(G,source):
    parent = [0]*len(G.nodes)
    distance = [sys.maxsize]*len(G.nodes)
    visited_set = [False]*len(G.nodes)
    parent[key_list.index(source)]= -1
    distance[key_list.index(source)]=0
    prev_node = 0
    node = key_list.index(source)
    def get_minimum_distance(distance,visited_set,prev_node):
        idx = prev_node
        maxi = sys.maxsize
        for i in range(len(distance)):
            if visited_set[i]==False and distance[i]<maxi:
                maxi = distance[i]
                idx = i
        return idx
    for i in range(len(G.nodes)-1):
        prev_node = node
        node = get_minimum_distance(distance,visited_set,prev_node)
        visited_set[node]= True
        for j in list(G.neighbors(key_list[node_list.index(node)])):
            if (distance[node_list[key_list.index(j)]]>distance[node]+G[key_list[node_list.index(node)]][j]["d"]) and (visited_set[node_list[key_list.index(j)]]==False) and (distance[node_list[key_list.index(j)]]>0):
                distance[node_list[key_list.index(j)]] = distance[node]+G[key_list[node_list.index(node)]][j]["d"]
                parent[node_list[key_list.index(j)]]= node
    for i in np.argsort(distance):
        if distance[i]!=0 and distance[i]!= sys.maxsize:
            print("Min Distance from :"+ str(source)+ " to "+ str(key_list[i]) + ": "+ str(distance[i]))
    

In [10]:
a= dijstra(G,"Springfield, IL")

Min Distance from :Springfield, IL to Reading, PA: 72
Min Distance from :Springfield, IL to Scranton, PA: 93
Min Distance from :Springfield, IL to Ravenna, OH: 106
Min Distance from :Springfield, IL to Rochester, NY: 129
Min Distance from :Springfield, IL to Wausau, WI: 135
Min Distance from :Springfield, IL to Scottsbluff, NB: 159
Min Distance from :Springfield, IL to Traverse City, MI: 181
Min Distance from :Springfield, IL to Springfield, MO: 186
Min Distance from :Springfield, IL to Rock Springs, WY: 187
Min Distance from :Springfield, IL to Terre Haute, IN: 193
Min Distance from :Springfield, IL to Valdosta, GA: 199
Min Distance from :Springfield, IL to Stevens Point, WI: 217
Min Distance from :Springfield, IL to Saint Joseph, MI: 243
Min Distance from :Springfield, IL to Tulsa, OK: 247
Min Distance from :Springfield, IL to Twin Falls, ID: 250
Min Distance from :Springfield, IL to Trenton, NJ: 259
Min Distance from :Springfield, IL to Wheeling, WV: 262
Min Distance from :Springfie

In [11]:
def path(G, source, destination):
    distance_btw = np.ones((distance_matrix.shape[0],distance_matrix.shape[0]))*sys.maxsize
    for i in range(distance_matrix.shape[0]):
        for j in range(i):
            if distance_matrix[i][j]<=100:
                G.add_edge(key_list[node_list.index(i)],key_list[node_list.index(j)])
                G[key_list[node_list.index(i)]][key_list[node_list.index(j)]]["d"]=distance_matrix[i][j]
                distance_btw[i][j]=distance_matrix[i][j]
                distance_btw[j][i]=distance_matrix[i][j]
    row_values = np.sum(init_matrix, axis =1)
    col_values = np.sum(init_matrix, axis = 0)
    rv_zip = list(zip(row_values,[i for i in range(128)]))
    cv_zip = list(zip(col_values,[i for i in range(128)]))
    zero_values = set([j for i,j in rv_zip if i==0])
    zero_col = set([j for i,j in cv_zip if i ==0])
    get_edge = (zero_values.intersection(zero_col)) 
    for i in get_edge:
        idx = np.argsort(distance_matrix[i])
        G.add_edge(key_list[node_list.index(i)],key_list[node_list.index(idx[1])])
        G[key_list[node_list.index(i)]][key_list[node_list.index(idx[1])]]["d"]=distance_matrix[i][idx[1]]
        distance_btw[i][idx[1]]=distance_matrix[i][idx[1]]
        distance_btw[idx[1]][i]= distance_matrix[i][idx[1]]
        prev_matrix = distance_btw
    distance_btw = np.fill_diagonal(distance_btw,0)
    
    #main function starts.....
    for k in range(len(G.nodes)):
        new_matrix = prev_matrix.copy()
        for i in range(len(G.nodes)):
            for j in range(len(G.nodes)):
                if prev_matrix[i][j]> prev_matrix[i][k]+ prev_matrix[k][j]:
                    new_matrix[i][j]= prev_matrix[i][k]+ prev_matrix[k][j]
        prev_matrix = new_matrix
    print("The minimum distance from "+ str(source)+ " to "+ str(destination)+ "is : ", prev_matrix[key_list.index(source)][key_list.index(destination)])
    return prev_matrix

In [12]:
output = path(G,"Reading, PA", "Springfield, IL")

The minimum distance from Reading, PA to Springfield, ILis :  72.0


# Finding largest SP cost

In [13]:
largest_spcost = 0
for i in range(len(G.nodes)):
    for j in range(len(G.nodes)):
        if output[i][j]!= sys.maxsize and output[i][j]>largest_spcost:
            largest_spcost = output[i][j]
            source_node = i
            destination_node = j
print("The minimum distance from "+ str(key_list[source_node])+ " to "+ str(key_list[destination_node])+ "is : ", output[source_node][destination_node])

The minimum distance from Salt Lake City, UT to Regina, SAis :  1052.0
